In [ ]:
import os
import sys
import json
import datetime
import numpy as np
import skimage.draw
import cv2
from mrcnn.visualize import display_instances
import matplotlib.pyplot as plt

# Root directory of the project
ROOT_DIR = 'C:\\Users\\Documents\\MaskRCNN_Car damage_Multi Class\\'

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import model as modellib, utils

# Path to trained weights file
COCO_WEIGHTS_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

# Directory to save logs and model checkpoints, if not provided
# through the command line argument --logs
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")

dataset_dir = 'C:\\Users\\Documents\\MaskRCNN_Car damage_Multi Class\\dataset\\'


In [2]:
class CustomConfig(Config):
    """Configuration for training on the damage dataset.
    Derives from the base Config class and overrides some values.
    """
    # Give the configuration a recognizable name
    NAME = "Damage"

    # We use a GPU with 8GB memory, which can fit >1  image, but we use 1 image
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

    # Number of classes (including background)
    NUM_CLASSES = 1 + 2  # Car Background + Dent + scratch

    # Number of training steps per epoch
    STEPS_PER_EPOCH = 100

    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.9


In [3]:
config=CustomConfig()

In [30]:
class DamageDataset(utils.Dataset):
    
    def load_custom(self, dataset_dir, subset):
        """Load a subset of the dataset.
        dataset_dir: Root directory of the dataset.
        subset: Subset to load: train or val
        """
        # Add classes. We have only one class to add.
        self.add_class("Damage", 1, "Dent")
        self.add_class("Damage", 2, "Scratch")

        # Train or validation dataset?
        assert subset in ["train", "val"]
        dataset_dir = os.path.join(dataset_dir + subset)

        # We only about the x and y coordinates of each region
        annotations1 = json.load(open(os.path.join(dataset_dir + '\\' + "via_region_data.json"),'r',encoding="utf8",errors='ignore'))
        
        annotations = list(annotations1.values())  # don't need the dict keys

        # The VIA tool saves images in the JSON even if they don't have any
        # annotations. Skip unannotated images.
        annotations = [a for a in annotations if a['regions']]

        # Add images
        for a in annotations:
            # Get the x, y coordinaets of points of the polygons that make up
            # the outline of each object instance. There are stores in the
            # shape_attributes 
            polygons = [r['shape_attributes'] for r in a['regions'].values()]
            objects = [s['region_attributes'] for s in a['regions'].values()]
            
            num_ids = []
            for n in objects:
                try:
                    if n['Damage'] == 'Dent':
                        num_ids.append(1)
                    elif n['Damage'] == 'Scratch':
                        num_ids.append(2)
                  
                except:
                    pass
            # load_mask() needs the image size to convert polygons to masks.
            # Unfortunately, VIA doesn't include it in JSON, so we must read
            # the image. This is only managable since the dataset is tiny.
            image_path = os.path.join(dataset_dir, a['filename'])
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]

            self.add_image(
                "Damage",  # for a single class just add the name here
                image_id=a['filename'],  # use file name as a unique image id
                path=image_path,
                width=width, height=height,
                polygons=polygons,
                num_ids=num_ids)

    def load_mask(self, image_id):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        # If not a Damage dataset image, delegate to parent class.
        image_info = self.image_info[image_id]
        if image_info["source"] != "Damage":
            return super(self.__class__, self).load_mask(image_id)
        
        info = self.image_info[image_id]
        # Convert polygons to a bitmap mask of shape
        # [height, width, instance_count]
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)

        for i, p in enumerate(info["polygons"]):
            # Get indexes of pixels inside the polygon and set them to 1
            rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])
            mask[rr, cc, i] = 1
       
        # Map class names to class IDs.
        num_ids = info['num_ids']
        #return mask.astype(np.bool), num_ids.astype(np.int32)
        return mask, np.array(num_ids)

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "Damage":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)


In [14]:
def train(model):
    """Train the model."""
    # Training dataset.
    dataset_train = DamageDataset()
    dataset_train.load_custom(dataset_dir, "train")
    dataset_train.prepare()

    # Validation dataset
    dataset_val = DamageDataset()
    dataset_val.load_custom(dataset_dir, "val")
    dataset_val.prepare()

    # *** This training schedule is an example. Update to your needs ***
    # Since we're using a very small dataset, and starting from
    # COCO trained weights, we don't need to train too long. Also,
    # no need to train all layers, just the heads should do it.
	# Download mask_rcnn_coco.h5 weights before starting the training
    print("Training network heads")
    model.train(dataset_train,dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=15,
                layers='heads')



In [9]:
print("Loading Mask R-CNN model...")
model = modellib.MaskRCNN(mode="training", config=config, model_dir='./')

Loading Mask R-CNN model...
Instructions for updating:
Colocations handled automatically by placer.


In [10]:
weights_path = COCO_WEIGHTS_PATH

# Load weights
print("Loading weights ", weights_path)

Loading weights  C:\Users\abhis\Documents\MaskRCNN_Car damage_Multi Class\mask_rcnn_coco.h5


In [11]:
model.keras_model.summary()

Model: "mask_rcnn"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_image (InputLayer)        (None, None, None, 3 0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, None, None, 3 0           input_image[0][0]                
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
bn_conv1 (BatchNorm)            (None, None, None, 6 256         conv1[0][0]                      
__________________________________________________________________________________________

__________________________________________________________________________________________________
bn4s_branch2c (BatchNorm)       (None, None, None, 1 4096        res4s_branch2c[0][0]             
__________________________________________________________________________________________________
add_26 (Add)                    (None, None, None, 1 0           bn4s_branch2c[0][0]              
                                                                 res4r_out[0][0]                  
__________________________________________________________________________________________________
res4s_out (Activation)          (None, None, None, 1 0           add_26[0][0]                     
__________________________________________________________________________________________________
res4t_branch2a (Conv2D)         (None, None, None, 2 262400      res4s_out[0][0]                  
__________________________________________________________________________________________________
bn4t_branc

In [12]:
model.load_weights(weights_path, by_name=True, exclude=[
            "mrcnn_class_logits", "mrcnn_bbox_fc",
            "mrcnn_bbox", "mrcnn_mask"])

In [32]:
train(model)

Training network heads

Starting at epoch 0. LR=0.001

Checkpoint Path: ./damage20200412T1345\mask_rcnn_damage_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask_conv4       (TimeDistributed)
mrcnn_mask_b

C:\Users\abhis\Anaconda3\envs\tensorflow2\lib\site-packages\tensorflow\python\ops\gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/15
100/100 [==============================] - 113s 1s/step - loss: 1.0580 - rpn_class_loss: 0.0251 - rpn_bbox_loss: 0.2082 - mrcnn_class_loss: 0.0970 - mrcnn_bbox_loss: 0.3230 - mrcnn_mask_loss: 0.4048 - val_loss: 1.6696 - val_rpn_class_loss: 0.0188 - val_rpn_bbox_loss: 0.4888 - val_mrcnn_class_loss: 0.0916 - val_mrcnn_bbox_loss: 0.6212 - val_mrcnn_mask_loss: 0.4491
Epoch 2/15
100/100 [==============================] - 104s 1s/step - loss: 0.9973 - rpn_class_loss: 0.0214 - rpn_bbox_loss: 0.1868 - mrcnn_class_loss: 0.0906 - mrcnn_bbox_loss: 0.3252 - mrcnn_mask_loss: 0.3733 - val_loss: 1.6230 - val_rpn_class_loss: 0.0180 - val_rpn_bbox_loss: 0.4882 - val_mrcnn_class_loss: 0.1021 - val_mrcnn_bbox_loss: 0.6071 - val_mrcnn_mask_loss: 0.4077
Epoch 3/15
100/100 [==============================] - 102s 1s/step - loss: 0.8674 - rpn_class_loss: 0.0201 - rpn_bbox_loss: 0.1797 - mrcnn_class_loss: 0.0767 - mrcnn_bbox_loss: 0.2516 - mrcnn_mask_loss: 0.3392 - val_loss: 1.7088 - val_rpn_class_l

In [33]:
model.keras_model.save_weights('C:\\Users\\Documents\\MaskRCNN_Car damage_Multi Class\\CarDamageMultiClass_mask_rcnn_model_12042020.h5')